In [1]:
import os
import pandas as pd
import numpy as np

path = '/Users/zhangyuan/Google Drive/Files_PNC_Analysis'

# load subject info
filename = 'SubInfo_2018.txt'
sub = pd.read_csv(os.path.join(path, filename), header=0, delimiter='\t')
print("sub shape: " + str(sub.shape))
print("The first 3 columns are: " + str(sub.columns[0:3]))
sub.head(3)

num_sub = sub.shape[0]

sub shape: (759, 10)
The first 3 columns are: Index(['Subj', 'Age', 'Sex'], dtype='object')


In [2]:
# emotion condition (classes)
emotion = ['fear','anger','sad','happy'] # this is a list
#print(emotion)

# load connectivity into dictionary conn
path = '/Users/zhangyuan/Google Drive/Files_PNC_Analysis'
path = path +'/BS_Conn_50ROIs_2018/Original Data'
#print(path)

df = {} # save connectivity under Fear, Anger, Sad, and Happy

for i in range(len(emotion)):
    filename = 'bs_conns_' + emotion[i] + '.txt'  
    #print(filename)
    conn = pd.read_csv(os.path.join(path, filename), header=0, delimiter='\t')
    # check the shape
    #print(conn.shape)
    # get name of the last unused column
    col = conn.columns[-1]
    #print("last column: " + str(col))
    # drop the last unused column
    conn = conn.drop(str(col),1)
    # check the shape
    #print(conn.shape)
    df[str(emotion[i])] = pd.concat([sub, conn], axis=1) #concat
    

In [3]:
# Data check
print(df.keys())
print(df[str(emotion[0])].shape)
df[str(emotion[0])].head(3)

dict_keys(['fear', 'anger', 'sad', 'happy'])
(759, 1236)


,Subj,Age,Sex,Race,GScore,GGroup,DScore,DGroup,Motion,Agebin,...,VLPFC.R_DLPFC.L,VLPFC.R_DLPFC.R,VLPFC.R_DLPFCm.L,VLPFC.R_DLPFCm.R,DLPFC.L_DLPFC.R,DLPFC.L_DLPFCm.L,DLPFC.L_DLPFCm.R,DLPFC.R_DLPFCm.L,DLPFC.R_DLPFCm.R,DLPFCm.L_DLPFCm.R
0,600009963128,9.666667,0,0,0,0,0,0,0.149664,Children,...,0.4722,0.9470,0.7317,1.2809,0.8010,0.0175,0.1136,0.2156,0.5137,1.2603
1,600018902293,15.583333,0,0,12,1,4,1,0.059930,Adolescents,...,0.3621,0.3730,0.5669,0.5315,1.1184,1.1773,1.0380,1.1589,0.6858,1.2571
2,600020364885,17.750000,1,0,2,1,0,0,0.056563,Adolescents,...,0.7979,0.4853,0.3248,0.6449,1.0761,0.0941,0.2069,-0.0811,0.1089,0.9859


In [4]:
# combine four emotion data
data = pd.concat([df["fear"],df["anger"],df["sad"],df["happy"]], axis=0)
print(data.shape)

(3036, 1236)


In [5]:
# replace missing values with column mean
def replace_nan(X):
    col_mean = np.nanmean(X, axis=0) # obtain column means
    ind = np.where(np.isnan(X)) # find the position of missing values
    X[ind] = np.take(col_mean, ind[1]) # replace nans with colmeans
    
    return X

In [6]:
# Prepare data for emotion classification

# Features
X = data.drop(columns = sub.columns) # drop subID, age, group etc.
#X.head(3)
X = X.drop(columns = X.columns[0]) # drop subID
#X.head(3)
X = X.values # convert pandas dataframe to numpy array
# print(X.shape)
#print(type(X))
X = replace_nan(X) # deal with missing values in numpy array

# Labels
y = np.array([1,2,3,4]) 
y = np.repeat(y, num_sub)

# Groups
# Group info that will be used to do train-test split
# data belongs to the same subject should not be splited into different folds
G = np.array(range(num_sub)) +1
G = np.tile(G, 4).T
#print(G[0,759:789])
print('X shape: ' + str(X.shape) + '\ty shape: ' + str(y.shape) + '\tG shape: ' + str(G.shape))


X shape: (3036, 1225)	y shape: (3036,)	G shape: (3036,)


In [7]:
# import packages for svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GroupShuffleSplit

In [8]:
# train-test split for 10 fold cross validation; 
# samples that belong to the same subject will not be
# split into different folds
def get_cv(X, y, G):
    cv = GroupShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 42)
    cv = cv.split(X, y, G)
    return cv

# train_inds, test_inds = next(cv) 

In [9]:
# define my SVC model (data normalization, SVC)
def mySVC(gamma, C):
    return Pipeline([
        ("std_scaler", StandardScaler()),
        ("svc", SVC(kernel="rbf", gamma=gamma, C=C))
    ])

In [10]:
# model = mySVC(gamma=1, C=0.1)
# model.fit(X_train,y_train)
# model.score(X_test, y_test)

In [11]:
def evaluation(X, y, G, gamma, C):
    pipe = mySVC(gamma=gamma, C=C) 
    cv = get_cv(X, y, G)
    #print("type(cv) = {}".format(type(cv)))
    
    results = cross_validate(pipe, X, y, scoring=['accuracy'], cv=cv,
                             verbose=1, return_train_score=True,
                             n_jobs=-1)
    
    return results

In [13]:
#results = evaluation(X,y,G,gamma=1,C=0.1)
#print("Training score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['train_accuracy']),
#                                                         np.std(results['train_accuracy'])))
#print("Validation score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']),
#                                                           np.std(results['test_accuracy'])))

In [15]:
C_range = np.exp2(np.array(range(-5,5))) #np.logspace(-3, 2, 6)
gamma_range = np.exp2(np.array(range(-10,1)))
models = []
for C in C_range:
    for gamma in gamma_range:
        results = evaluation(X,y,G,gamma,C)
        models.append((C,gamma,results))
        print("Training score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['train_accuracy']),
                                                         np.std(results['train_accuracy'])))
        print("Validation score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']),
                                                           np.std(results['test_accuracy'])))

Training score accuracy: 0.500 +- 0.010
Validation score accuracy: 0.303 +- 0.016


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.9min finished


Training score accuracy: 0.938 +- 0.011
Validation score accuracy: 0.306 +- 0.020


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.7min finished


Training score accuracy: 1.000 +- 0.000
Validation score accuracy: 0.272 +- 0.017


...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished


Training score accuracy: 1.000 +- 0.000
Validation score accuracy: 0.249 +- 0.012


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished


Training score accuracy: 1.000 +- 0.000
Validation score accuracy: 0.250 +- 0.000


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished
